In [1]:
import sys
import warnings
import datacube
from datacube.storage import masking
from datacube.helpers import ga_pq_fuser
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns
import pandas as pd
import xarray as xr
from datacube.utils import geometry
from datacube.utils.geometry import CRS
from datacube.helpers import write_geotiff
import os

sys.path.append('../../10_Scripts')
import SpatialTools
import gdal
import DEADataHandling
import DEAPlotting

# Connect to datacube database
dc = datacube.Datacube(app='Snow monitoring with Landsat')


output='/g/data/r78/jt7365/dea-notebooks/SnowOFS/results/Tas/1200/phen/cm/'
year='2016_2018_'
elev = 1200

# Set up centre of area to analyse, and a buffer in metres around this centrepoint. Generally,
# aim to keep the buffer size to less than 10,000m to prevent having memory errors


In [6]:
lat, lon, buffer_m, buffer_n = -41.85, 146.38, 8000, 8000 #tas cradle mountain/mt ossa region
#lat, lon, buffer_m, buffer_n = -41.87, 146, 8000, 8000 # tas mt ossa
#lat, lon, buffer_m, buffer_n = -41.56728, 147.67106, 10000, 10000 #tas ben lomond
#lat, lon, buffer_m, buffer_n = -36.3, 148.35, 30000, 40000 #nsw

# Set range of time to return data from both Landsat and Sentinel 2
time_range = ('2016-01-01', '2018-12-31')

resolution = (-25, 25)

landsat_goodquality_prop = 0.3

ndsi_threshold = 0.2
ndvi_threshold = 0.3
ndwi_threshold = 0.15
red_threshold = 1000


In [7]:
x, y = geometry.point(lon, lat, CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]
query = {'x': (x - buffer_m, x + buffer_m),
         'y': (y - buffer_m, y + buffer_m),
         'time': time_range,
         'crs': 'EPSG:3577',
         'output_crs': 'EPSG:3577',
         'resolution': resolution}

print('loading landsat_ds')
snow_mask = dict(                cloud_acca='no_cloud',
                                 #cloud_fmask='no_cloud',
                                 cloud_shadow_acca='no_cloud_shadow',
                                 #cloud_shadow_fmask='no_cloud_shadow',
                                 blue_saturated=False,
                                 green_saturated=False,
                                 red_saturated=False,
                                 nir_saturated=False,
                                 swir1_saturated=False,
                                 swir2_saturated=False,
                                 contiguous=True)

landsat_ds = DEADataHandling.load_clearlandsat(dc=dc,
                                               product = 'nbar',
                                               query=query,
                                               mask_dict=snow_mask,
                                               sensors=['ls5','ls7','ls8'],
                                               bands_of_interest=['red', 'green', 'blue','nir', 'swir1'],
                                               masked_prop=landsat_goodquality_prop,
                                               mask_pixel_quality=True,
                                               ls7_slc_off=True)

# file = open(output+'nofimages.txt','a')
# file.write(year+', '+str(len(landsat_ds.time))+'\n')
# file.close


loading landsat_ds
Loading ls5
    Skipping ls5; no valid data for query
Loading ls7
    Loading 22 filtered ls7 timesteps
Loading ls8
    Loading 24 filtered ls8 timesteps
Combining and sorting ls7, ls8 data
    Replacing invalid -999 values with NaN (data will be coerced to float64)


In [11]:

ndsi_ds = (landsat_ds.green -landsat_ds.swir1) / (landsat_ds.green + landsat_ds.swir1)
ndvi_ds = (landsat_ds.nir - landsat_ds.red) / (landsat_ds.nir + landsat_ds.red)
ndwi_ds = (landsat_ds.green - landsat_ds.nir)/(landsat_ds.green + landsat_ds.nir)

valid_ds = np.isfinite(landsat_ds.swir1)

snow_ds1 = ndsi_ds > ndsi_threshold
noveg_ds = ndvi_ds < ndvi_threshold
nowater_ds = ndwi_ds < ndwi_threshold
red_ds = landsat_ds.red > red_threshold
snow_ds = noveg_ds & snow_ds1 & nowater_ds & red_ds
#snow_ds = noveg_ds & snow_ds1


x, y = geometry.point(lon, lat, CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]
query1 = {'x': (x - buffer_m, x + buffer_m),
         'y': (y - buffer_n, y + buffer_n),
         #'time': time_range,
         'crs': 'EPSG:3577',
         'output_crs': 'EPSG:3577',
         'resolution': resolution}

# Load the DEM data
dem_ds = dc.load(product='srtm_dem1sv1_0', **query1)
dem_elev = dem_ds.dem > elev


In [12]:
# snow_elev = snow_ds.where(dem_elev.squeeze(dim='time'))
# valid_elev = valid_ds.where(dem_elev.squeeze(dim='time'))
# snow_prop = snow_elev/valid_elev

c=list(range(len(snow_ds.time)))
snow_prop_elev=[]
for i in c:
    d = valid_ds.isel(time=i).values * dem_elev
    f = snow_ds.isel(time=i).values * dem_elev
    n = np.count_nonzero(d.values)
    if n == 0:
        n = n+1
    snow =  np.count_nonzero(f.values) / n
    snow_prop_elev.append(snow)



In [13]:
# mask ndsi dataset with dem_1800, leaving values over 1800 m
# ndsi_elev = ndsi_ds.where(dem_elev.squeeze(dim='time'))
# calculate ndsi mean of valid pixels over 1800 m
# mean_ndsi_elev = ndsi_elev.mean(dim=('x', 'y'))


# df = pd.DataFrame({'time':snow_ds.time, 'snow_prop_1800':snow_prop_1800, 'pixel_prop':prop_, 'pixel_over_005':prop_rel})
# df.to_csv(output+years+'NEWmask_frequency_prop.csv')

# df = pd.DataFrame({'dayofyear':snow_ds['time.dayofyear'], 'proportion':snow_ds.values, 'year':ndsi_ds['time.year']})
# df.to_csv(output+years+'phenology_freq_1800.csv')

df = pd.DataFrame({'dayofyear':snow_ds['time.dayofyear'], 'snow_prop_elev':snow_prop_elev, 'year':snow_ds['time.year']})
df.to_csv('phenology_freq_'+str(elev)+'.csv')
